# GEMS PedTools Example Usage
Below is a simple example that illustrates how to access data in the PedTools database.
### Set up an HTTP client using Python's request library
We use a `Session` object to store our API key and automatically include it in the header for each request.

Note that we have a `api_key.py` file in the Exchange-Notebooks directory. The file contains only the below line.
```
api_key = 'SECRET'
```

In [7]:
import json
import pandas as pd
from requests import Session
from urllib.parse import quote
import sys
sys.path.append('..')
from api_key import api_key

s = Session()
s.headers.update({'apikey': api_key})

base = 'https://exchange-1.gems.msi.umn.edu/pedtools/v1'

### Find info on a variety of interest
We use the `/{variety}` endpoint to obtain all recorded info on a variety, in this case 'TURKEY'.

In [8]:
params = {'pedigree_depth': 5}
variety = quote('TURKEY')
res = s.get(f'{base}/{variety}', params=params)
df = pd.json_normalize(res.json())
df

,preferred_name,crop_name,is_landrace,backcross_depth,selfing_count,market_class,release_date,developer,parentage,aliases,...,mother.crop_name,mother.is_landrace,mother.backcross_depth,mother.selfing_count,mother.market_class,mother.release_date,mother.developer,mother.parentage,mother.aliases,mother
0,TURKEY GID:135,wheat,False,0,2,None,None,None,None,"[{'name': '135', 'type': 'germplasm_bank_id'},...",...,wheat,False,0.0,1.0,NaN,NaN,NaN,NaN,"[{'name': '3700', 'type': 'germplasm_bank_id'}...",NaN
1,TURKEY GID:10509,wheat,False,0,0,None,None,None,None,"[{'name': '10509', 'type': 'germplasm_bank_id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cool two items were returned. Can I see all the aliases for the first one?

In [9]:
df['aliases'][0]

[{'name': '135', 'type': 'germplasm_bank_id'},
 {'name': 'CWI51783', 'type': 'germplasm_bank_accession_id'},
 {'name': 'P.1066-?-?', 'type': 'selection_history'},
 {'name': 'TK', 'type': 'cross_abbreviation'},
 {'name': 'TURKEY', 'type': 'cross_name'}]

### Let's check another entry, and retrieve its pedigree to a depth of 5 (great-great-great grandparents).

In [10]:
params = {'pedigree_depth': 5}
variety = quote('SANDPIPER')
#variety = quote('###33476')
res = s.get(f'{base}/{variety}', params=params)
df = pd.json_normalize(res.json())
df

,preferred_name,crop_name,is_landrace,backcross_depth,selfing_count,market_class,release_date,developer,parentage,aliases,...,father.preferred_name,father.crop_name,father.is_landrace,father.backcross_depth,father.selfing_count,father.market_class,father.release_date,father.developer,father.parentage,father.aliases
0,SANDPIPER,wheat,False,0,0,None,None,None,None,"[{'name': '4046', 'type': 'germplasm_bank_id'}...",...,FLAMINGO,wheat,False,0,6,None,None,None,None,"[{'name': '-B14263.05-0T-0L-28T-0L-?', 'type':..."


### One match. Let's retrieve its pedigree.

In [12]:
df['pedigree'][0]

'###123561 / STEWART 63 /2/ 980 GID:980 / WELLS /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /4/ POLONICUM PI185309 /2/ ###250171 / TEHUACAN 60 /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /5/ II23584 GID:417 (417) F6 /2/ ###6385 (###6385) F2 /3/ ###3926 (###3926) F1 (II23584 GID:417 (417) F6 /2/ ###6385 (###6385) F2 /3/ ###3926 (###3926) F1) F6'

### How do I find the Coefficient of Parentage (COP) between any arbitrary pair of varieties?
We use the `/{variety1}/{variety2}/cop` endpoint to obtain the COP between two varieties, variety1 and variety2.

In [13]:
var1 = quote('###33476')
var2 = quote('MEDITERRANEAN')
res = s.get(f'{base}/{var1}/{var2}/cop')
pd.json_normalize(res.json())

,cop
0,0.125
